In [ ]:
# unrar is proprietary
!echo "deb http://deb.debian.org/debian buster main contrib non-free" >> /etc/apt/sources.list
!apt-get update > /dev/null

# Install unrar 
! apt-get update && apt-get install -y unrar > /dev/null

! pip install rarfile > /dev/null

W: Target Packages (main/binary-amd64/Packages) is configured multiple times in /etc/apt/sources.list:2 and /etc/apt/sources.list:7
W: Target Packages (main/binary-all/Packages) is configured multiple times in /etc/apt/sources.list:2 and /etc/apt/sources.list:7
W: Target Packages (main/binary-amd64/Packages) is configured multiple times in /etc/apt/sources.list:2 and /etc/apt/sources.list:8
W: Target Packages (main/binary-all/Packages) is configured multiple times in /etc/apt/sources.list:2 and /etc/apt/sources.list:8
W: Target Packages (contrib/binary-amd64/Packages) is configured multiple times in /etc/apt/sources.list:7 and /etc/apt/sources.list:8
W: Target Packages (contrib/binary-all/Packages) is configured multiple times in /etc/apt/sources.list:7 and /etc/apt/sources.list:8
W: Target Packages (non-free/binary-amd64/Packages) is configured multiple times in /etc/apt/sources.list:7 and /etc/apt/sources.list:8
W: Target Packages (non-free/binary-all/Packages) is configured multiple

In [ ]:
# !git clone https://github.com/TengMichael/battery-charging-data-of-on-road-electric-vehicles.git
! pip install statsmodels ipywidgets dask dask-ml > /dev/null


[notice] A new release of pip is available: 23.1.2 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [ ]:
import numpy as np
import scipy
from scipy import stats as stats
import math
import pandas as pd
import matplotlib.pyplot as plt
import ipywidgets
from ipywidgets import interact, interactive, fixed, interact_manual
%matplotlib inline
import seaborn as sns
import statsmodels.api as sm
import statsmodels.formula.api as smf
import glob
import dask.dataframe as dd
from dask_ml.linear_model import LinearRegression
import dask






/root/venv/lib/python3.11/site-packages/dask/dataframe/_pyarrow_compat.py:23: UserWarning: You are using pyarrow version 13.0.0 which is known to be insecure. See https://www.cve.org/CVERecord?id=CVE-2023-47248 for further details. Please upgrade to pyarrow>=14.0.1 or install pyarrow-hotfix to patch your current version.
  warnings.warn(


Step 1: Load

In [ ]:
convert = False
if convert:
    import os
    import rarfile
    # import pandas as pd
    from io import BytesIO, BufferedReader

    # Directory containing your RAR files
    rar_files_dir = '/work/battery-charging-data-of-on-road-electric-vehicles/'

    # List all RAR files in the directory
    rar_files = [os.path.join(rar_files_dir, f) for f in os.listdir(rar_files_dir) if f.endswith('.rar')]

    # Initialize an empty list to store DataFrames
    dataframes = []

    def vin_from_filename(filename: str):
        return filename.strip('.rar').split('#')[1]


    from dask.base import normalize_token

    @normalize_token.register(rarfile.PipeReader)
    def normalize_token(x: rarfile.PipeReader):
        return x.filename

    # Process each RAR file
    for rar_file in rar_files:
        with rarfile.RarFile(rar_file) as rf:
            # Extract each file in the RAR archive
            for info in rf.infolist():
                # Read file content into a buffer
                print(info.filename)
                with rf.open(info.filename) as f:
                    # Read the CSV file into a DataFrame
    
                    # NOTE: normalize token is needed
                    df = pd.read_csv(
                        f,
                        dtype={
                            'Unnamed: 0': 'int32',
                            'soc': 'float32',
                            'pack_voltage (V)': 'float32',
                            'charge_current (A)': 'float32',
                            'max_cell_voltage (V)': 'float32',
                            'min_cell_voltage (V)': 'float32',
                            'max_temperature (℃)': 'int32',
                            'min_temperature (℃)': 'int32',
                            'available_energy (kw)': 'float32',
                            'available_capacity (Ah)': 'float32'
                        }
                    )
                    vin = vin_from_filename(rar_file)
                    # Append the DataFrame to the list
                    # dataframes.append(df)
                    df = dd.from_pandas(df)
                    df.to_parquet(f'data/data-v3-{vin}.parquet')

    # break

# Concatenate all DataFrames into a single DataFrame
# combined_df = dd.concat(dataframes, ignore_index=True)

# Display the combined DataFrame
# print(combined_df.head())


In [ ]:
def clean_data(data, session_offset):

    # data['record_time'] = pd.to_datetime(data['record_time'], format='%Y%m%d%H%M%S')
    data['record_time'] = dd.to_datetime(data['record_time'], format='%Y%m%d%H%M%S')
    # data.compute()  # Do not call compute until the results are smaller
    print(data['record_time'].head())  # Here we only output 5 row of data;
    # so computations are done only on around 5 rows; we will not run out of memory in this case 
    print("\nUpdated Data Types:\n", data.dtypes)

    df = data

    # Define the criteria for a new session (e.g., time gap or change in charging current)
    df['time_diff'] = df['record_time'].diff().dt.total_seconds().fillna(0)

    # Define session change if time difference is greater than a threshold (e.g., 10 minutes)
    threshold = 600  # seconds
    df['new_session'] = (df['time_diff'] > threshold).cumsum() + session_offset

    # Determine charging type
    # Assuming fast charging (DC) current is greater than a certain threshold
    dc_current_threshold = -40  # This threshold should be adjusted based on real data characteristics

    # Use the minimum charge current to determine charge type
    session_types = df.groupby('new_session')['charge_current (A)'].min().apply(
        lambda x: 'DC' if x < dc_current_threshold else 'AC', 
        meta=('charging_type', 'str')
    )  #.compute()

    print(session_types.head())

    # # Convert the session_types back to a Dask DataFrame for merging
    # session_types_ddf = dd.from_pandas(session_types, npartitions=4)

    session_types_ddf = session_types

    # Merge the charging type back to the original Dask DataFrame
    ddf = df.merge(
        session_types_ddf.to_frame(name='charging_type'), 
        left_on='new_session', 
        right_index=True,
        how='left'
    )
    print(ddf.head())

    ddf['new_session'] = ddf['new_session'].astype(int)

    return ddf

In [ ]:

import dask.dataframe as dd
import os

# Directory containing your Parquet files
parquet_files_dir = '/work/data/original-by-vin'

# List all Parquet files in the directory
parquet_files = [
    os.path.join(parquet_files_dir, f) 
    for f in os.listdir(parquet_files_dir) 
    if f.endswith('.parquet') and f.startswith('data-v3-')
]

# Function to read a Parquet file and add a filename column
def read_parquet_with_filename(filepath):
    # Read the Parquet file
    df = dd.read_parquet(filepath)
    # Add a column with the filename
    df['source_file'] = str(os.path.basename(filepath))
    df = df.drop(columns=['Unnamed: 0'])
    return df

# Read all Parquet files and add the filename column
# Read first 5 vehicles
ddf_list = [
    read_parquet_with_filename(f) for f in parquet_files
]  # set to ":1" to reduce computation overhead

cleaned_ddf_list = [
    clean_data(ddf_list[i], session_offset=i*(10**8))
    for i in range(len(ddf_list))
]

# Concatenate all DataFrames into a single DataFrame
# data = combined_ddf = dd.concat(cleaned_ddf_list, axis=0)

          record_time        soc  pack_voltage (V)  charge_current (A)  \
0 2019-07-26 20:02:35  27.200001        328.200012          -52.200008   
1 2019-07-26 20:02:43  27.600000        328.500000          -52.200008   
2 2019-07-26 20:02:51  27.600000        328.600006          -52.200008   
3 2019-07-26 20:02:59  27.600000        328.600006          -52.200008   
4 2019-07-26 20:03:07  27.600000        328.799988          -52.200008   

   max_cell_voltage (V)  min_cell_voltage (V)  max_temperature (℃)  \
0                 3.656                 3.640                   41   
1                 3.663                 3.645                   41   
2                 3.665                 3.647                   41   
3                 3.666                 3.649                   41   
4                 3.666                 3.649                   41   

   min_temperature (℃)  available_energy (kw)  available_capacity (Ah)  \
0                   38                  12.40               

Step 2: Summary Statistics

In [ ]:
# print("Data Shape:", data.shape)
# print("\nData Types:\n", data.dtypes)
# print("\nSummary Statistics:\n", data.describe())

NOTES
Unnamed: 0: An index column ranging from 0 to 856,191.
record_time: Represents timestamps, ranging over a considerable period.
soc: The state of charge ranges from 0 to 100%, with a mean of approximately 64%.
pack_voltage (V): The voltage ranges from 312.8V to 386.2V, with a mean of around 352.09V.
charge_current (A): The current ranges from -400.02A to 12.10A, with a mean of -52.96A, indicating that the dataset primarily records discharging events.
max_cell_voltage (V) and min_cell_voltage (V): The cell voltages range from 0 to 4.264V and 4.241V, respectively.
max_temperature (℃) and min_temperature (℃): Temperatures range from 0 to 49℃ and 0 to 48℃, respectively, with means around 33.93℃ and 32.03℃.
available_energy (kw): The energy ranges from 0 to 45.29kW, with a mean of 27.69kW.
available_capacity (Ah): The capacity ranges from 0 to 136.16Ah, with a mean of 83.27Ah.

Step 3: Missing values

NOTES
No missing values!!!

Step 4: Data Type Conversion

In [ ]:
for df in cleaned_ddf_list:
    print(df.head())

          record_time        soc  pack_voltage (V)  charge_current (A)  \
0 2019-07-24 23:00:08  51.599998        338.600006          -70.399986   
1 2019-07-24 23:00:16  51.599998        339.100006          -70.399986   
2 2019-07-24 23:00:24  51.599998        339.399994          -70.399986   
3 2019-07-24 23:00:32  52.000000        339.700012          -70.500000   
4 2019-07-24 23:00:40  52.000000        339.899994          -70.399986   

   max_cell_voltage (V)  min_cell_voltage (V)  max_temperature (℃)  \
0                 3.786                 3.747                   35   
1                 3.791                 3.752                   35   
2                 3.795                 3.756                   35   
3                 3.799                 3.759                   35   
4                 3.801                 3.762                   35   

   min_temperature (℃)  available_energy (kw)  available_capacity (Ah)  \
0                   33              23.350000               

KeyboardInterrupt: 

In [ ]:
# for df in cleaned_ddf_list:
#     # print(df.head())
#     vin = df['source_file'][0].compute().split('-')[-1].split('.')[0]
#     print(f'outputting for {vin}')
#     df.to_parquet(f'data/by-vin-v11/{vin}.parquet')

from dask import delayed

def session_ddf(ddf: dd.DataFrame):
    """
    Agregate by sessions 
    """
        
    df = ddf.groupby('new_session').agg(
        start_soc=('soc', 'first'), 
        end_soc=('soc', 'last'),
        start_capacity=('available_capacity (Ah)', 'first'),
        end_capacity=('available_capacity (Ah)', 'last'),
        start_time=('record_time', 'first'),
        end_time=('record_time', 'last'),
        source_file=('source_file', 'first'),
        charging_type=('charging_type', 'first'),
    )

    df['diff_soc'] = df['end_soc'] - df['start_soc']
    df['diff_capacity'] = df['end_capacity'] - df['start_capacity']
    df['duration'] = df['end_time'] - df['start_time']

    df['full_capacity'] = df['end_capacity'] / (df['end_soc'] / 100)
 
    return df

session_ddf_list = [
    session_ddf(cleaned_ddf)
    for cleaned_ddf in cleaned_ddf_list ]



In [ ]:
print(session_ddf(cleaned_ddf_list[0]).head())

             start_soc    end_soc  start_capacity  end_capacity  \
new_session                                                       
0            51.599998  98.000000       70.189987    133.220001   
1            18.400000  99.199997       25.270000    134.889999   
2            20.000000  97.199997       27.260000    132.580002   
3            35.200001  64.800003       47.880001     88.189987   
4            40.400002  75.199997       54.790001    102.559998   

                     start_time            end_time         source_file  \
new_session                                                               
0           2019-07-24 23:00:08 2019-07-25 00:01:14  data-v3-10.parquet   
1           2019-07-26 11:01:10 2019-07-26 12:40:10  data-v3-10.parquet   
2           2019-07-27 11:04:05 2019-07-27 13:21:10  data-v3-10.parquet   
3           2019-07-28 00:55:21 2019-07-28 02:24:41  data-v3-10.parquet   
4           2019-07-28 05:36:19 2019-07-28 06:38:13  data-v3-10.parquet   

    

In [ ]:
for df in session_ddf_list:
    source_file = df.head(1).iloc[0]['source_file']
    vin = source_file.split('-')[-1].split('.')[0]
    print(f'outputting for {vin}')
    df.to_parquet(f'data/session-by-vin-v16/{vin}.parquet')

    

outputting for 10
outputting for 1
outputting for 11
outputting for 12
outputting for 13
outputting for 14
outputting for 15
outputting for 16
outputting for 17
outputting for 18
outputting for 19
outputting for 2
outputting for 20
outputting for 3
outputting for 4
outputting for 5
outputting for 6
outputting for 7
outputting for 8
outputting for 9


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=119995d2-42f9-4679-a55c-cd52fa6a2421' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>